In this notebook we will be implementing a mean-squared regression and a neural network to determine the optimal aggresion level against a typical opponent. We assume that there are two players, one of which has an aggression rating of 'Default', or 0.5.

Training data has been simualted using create_training.ipynb and is stored in '/data.training_data.csv'.

To first run a regression, we use an XGBoost ((Extreme Gradient Boosting)) regression model which minimizes the loss function (mean squared error) when fitting the data. Afterwards we're able to determine the input which returns the highest output.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# import training data
df = pd.read_csv('data/training_data.csv')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1], df.iloc[:, 2], test_size=0.2, random_state=42)

# Create an XGBoost regression model
model = xgb.XGBRegressor()

# Reshape the data to fit the model's requirements
X_train = np.array(X_train).reshape(-1, 1)
X_test = np.array(X_test).reshape(-1, 1)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model using mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Generate a range of inputs
inputs = np.linspace(0, 1, num=1000).reshape(-1, 1)

# Predict the outputs for the inputs using the trained model
outputs = model.predict(inputs)

# Find the input with the highest predicted output
highest_output_index = np.argmax(outputs)
highest_output_input = inputs[highest_output_index][0]

print("Input with the highest output:", highest_output_input)


Mean Squared Error: 0.0003115182919382953
Input with the highest output: 0.997997997997998


This result seems to be inconsistent with our prior study. While we've certaintly established that higher aggression ratios (conservative approaches) tend to win at a higher rate, the model indicates that a .99 ratio is optimal, i.e. rarely ever executing any trades and/or buying houses/hotels. It seems that this model is too simple to fully fit and model Monopoly.

Thus, we'll then create a neural network that predicts the win rate of a aggression ratio to be used to perform gradient descent to optimize our output.

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

df = pd.read_csv('data/training_data.csv')

# Load the input-output pairs from the dataframe
inputs = df.iloc[:, 1].values
outputs = df.iloc[:, 2].values

# Split the data into training and validation sets
train_inputs = inputs[:800]
train_outputs = outputs[:800]
val_inputs = inputs[800:]
val_outputs = outputs[800:]

# Define the neural network architecture
model = Sequential()
model.add(Dense(64, input_dim=1, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(train_inputs, train_outputs, epochs=50, batch_size=200, validation_data=(val_inputs, val_outputs))

# Use the model to predict the output for a given input
input_to_predict = 0.5
predicted_output = model.predict(np.array([input_to_predict]))

Epoch 1/50
4/4 [==============================] - 1s 116ms/step - loss: 0.1798 - val_loss: 0.1291
Epoch 2/50
4/4 [==============================] - 0s 23ms/step - loss: 0.1031 - val_loss: 0.0675
Epoch 3/50
4/4 [==============================] - 0s 21ms/step - loss: 0.0519 - val_loss: 0.0291
Epoch 4/50
4/4 [==============================] - 0s 19ms/step - loss: 0.0217 - val_loss: 0.0105
Epoch 5/50
4/4 [==============================] - 0s 21ms/step - loss: 0.0088 - val_loss: 0.0061
Epoch 6/50
4/4 [==============================] - 0s 23ms/step - loss: 0.0072 - val_loss: 0.0088
Epoch 7/50
4/4 [==============================] - 0s 20ms/step - loss: 0.0097 - val_loss: 0.0114
Epoch 8/50
4/4 [==============================] - 0s 21ms/step - loss: 0.0109 - val_loss: 0.0107
Epoch 9/50
4/4 [==============================] - 0s 22ms/step - loss: 0.0094 - val_loss: 0.0081
Epoch 10/50
4/4 [==============================] - 0s 21ms/step - loss: 0.0069 - val_loss: 0.0056
Epoch 11/50
4/4 [===========

In this case, we use a feedforward neural network with multiple hidden layers. This code defines a feedforward neural network with two hidden layers, each with a ReLU activation function. The output layer has a linear activation function, which is appropriate for predicting continuous values. The model is trained using the mean squared error loss function and the Adam optimizer. After training, the model can be used to predict the output for a given input.

Now, we can define a gradient descent function to optimize our output.

In [3]:
def gradient_descent(guess):
    def objective(x):
        input_to_predict = x
        predicted_output = model.predict(np.array([input_to_predict]))
        return predicted_output[0][0]

    # Define the initial input, the learning rate, and the number of iterations
    x = guess
    learning_rate = 0.01
    num_iterations = 100

    # Store prior iterations to terminate gradient descent early
    iter_vals = []

    beta = 0.99
    
    # Optimize the input using Adam optimization
    for i in range(num_iterations):
        if len(iter_vals) >= 2 and abs(iter_vals[-1] - iter_vals[-2]) < 0.0005:
             break

        # Compute the gradient of the objective function using numerical differentiation
        gradient = (objective(x + 0.001) - objective(x)) / 0.001
       
        x += (learning_rate * gradient) * (beta)**i

        iter_vals.append(x)

        print(x)

    return x, objective(x)

print(gradient_descent(0.5))

1/1 [==============================] - 0s 42ms/step
0.5041002035140991
1/1 [==============================] - 0s 16ms/step
0.5081594049930572
1/1 [==============================] - 0s 18ms/step
0.5121791828274727
1/1 [==============================] - 0s 18ms/step
0.5161576061969996
1/1 [==============================] - 0s 22ms/step
0.5200962453328312
1/1 [==============================] - 0s 19ms/step
0.5239960649115455
1/1 [==============================] - 0s 18ms/step
0.5278557639626753
1/1 [==============================] - 0s 20ms/step
0.5316774215775336
1/1 [==============================] - 0s 19ms/step
0.5354608626162433
1/1 [==============================] - 0s 20ms/step
0.5392070137432762
1/1 [==============================] - 0s 23ms/step
0.5429151643053155
1/1 [==============================] - 0s 23ms/step
0.5465867670249205
1/1 [==============================] - 0s 21ms/step
0.5502216537173293
1/1 [==============================] - 0s 19ms/step
0.553819145456237
1/1 [==

This model seems to better represent our monopoly game, optimizing a player's win rate at roughy 0.76. This falls in line with our prior study on aggression.

Of course, 0.76 is only optimal in a two person game where the opposing player plays at exactly a 0.50 aggression ratio. If any of these assumptions are violated, we would need to create new trading data through simulations and adjust our models accordingly. However, this study provides the guidlines for these future inquires and can be easily manipulated to fit various differing parameters.